# 😷 Face Mask Detector (Ultra Light ⚡)

## 📋 Sobre esta Versão
Esta é uma versão otimizada para **tamanho extremo (< 400KB)**.
Diferente da versão padrão, esta **NÃO usa Transfer Learning** da ImageNet, pois para atingir tamanhos tão pequenos, precisamos usar uma arquitetura MobileNetV2 com `alpha=0.1`, para a qual não existem pesos públicos pré-treinados.

**Consequência**: O modelo será treinado **do zero** (`scratch`).

## 🎯 Fases do Projeto
1. **Preparação dos Dados**: Download e pré-processamento.
2. **Data Augmentation**: Essencial aqui, pois vamos aprender do zero.
3. **Treinamento Completo**: Uma única fase longa de treinamento.
4. **Exportação**: Conversão para TFLite (INT8).

---


## 1. 🛠️ Configuração Inicial


In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import kagglehub
import tensorflow as tf
from collections import Counter
from sklearn.metrics import confusion_matrix
import glob

# Utilitários do Keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Constantes e Hiperparâmetros
BATCH_SIZE = 32
IMG_SIZE = (64, 64) 
IMG_SHAPE = IMG_SIZE + (3,)
SEED = 42

print(f"Versão do TensorFlow: {tf.__version__}")


## 2. 📥 Download dos Dados (KaggleHub)


In [ ]:
print("⬇️ Baixando Dataset...")
path = kagglehub.dataset_download("omkargurav/face-mask-dataset")
DATA_DIR = os.path.join(path, "data")

train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    validation_split=0.2,
    subset='training',
    seed=SEED
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    validation_split=0.2,
    subset='validation',
    seed=SEED
)

class_names = train_dataset.class_names
print(f"🏷️ Classes: {class_names}")


## 3. 🎨 Visualização


In [ ]:
# Pega uma imagem crua para visualização
all_image_paths = glob.glob(os.path.join(DATA_DIR, "*", "*.jpg"))
if not all_image_paths: # Fallback se não achar jpg
    all_image_paths = glob.glob(os.path.join(DATA_DIR, "*", "*"))
    
random_image_path = random.choice(all_image_paths)


## 4. ⚙️ Pré-processamento e Augmentation


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
])

def preprocess_train(image, label):
    image = data_augmentation(image)
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

def preprocess_val(image, label):
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

train_dataset = train_dataset.map(preprocess_train, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
validation_dataset = validation_dataset.map(preprocess_val, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)


## 5. 🏗️ Construção do Modelo (Ultra-Leve)
Aqui está a mudança chave: `weights=None` e `alpha=0.1`.


In [ ]:
print("🏗️ Construindo MobileNetV2 do ZERO (Alpha=0.1)...")

# Base Ultra Leve
base_model = MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights=None,   # <--- Sem pesos pré-treinados
    alpha=0.1       # <--- Rede extremamente pequena (~10% do tamanho original)
)

# Como não tem pesos pré-treinados, treinamos TUDO desde o início
base_model.trainable = True

inputs = Input(shape=IMG_SHAPE)
x = base_model(inputs)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x) # Dropout menor pois a rede já é pequena e tem poucos params
outputs = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(
    optimizer=Adam(learning_rate=1e-3), # LR normal para começar do zero
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

model.summary()


## 6. 🚀 Treinamento (Scratch)
Treinamos por mais épocas (30-40) para compensar a falta de pré-treino.


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

print("\n🚀 Iniciando Treinamento Completo...")

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=40, # Mais épocas
    callbacks=[early_stop, reduce_lr]
)


## 7. 📊 Resultados


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Treino Acc')
plt.plot(val_acc, label='Validação Acc')
plt.legend()
plt.title('Acurácia')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Treino Loss')
plt.plot(val_loss, label='Validação Loss')
plt.legend()
plt.title('Loss')
plt.show()

# Diagnóstico de Overfitting
final_train_acc = acc[-1]
final_val_acc = val_acc[-1]
gap = final_train_acc - final_val_acc

print(f"\n📊 Diagnóstico Final:")
print(f"   Acurácia Treino: {final_train_acc*100:.2f}%")
print(f"   Acurácia Validação: {final_val_acc*100:.2f}%")
print(f"   GAP: {gap*100:.2f}%")

if gap > 0.10:
    print("⚠️ ALERTA: Possível Overfitting (Gap > 10%)")
else:
    print("✅ Modelo Generalizando Bem!")


## 8. 🧪 Teste Externo (Prova Real)
Validamos o modelo com um dataset TOTALMENTE NOVO que o modelo nunca viu.


In [ ]:
print("\n🌍 Baixando Dataset de Teste Externo...")
test_path = kagglehub.dataset_download("belsonraja/face-mask-dataset-with-and-without-mask")
TEST_DIR = os.path.join(test_path, "facemask-dataset", "dataset")

test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Aplica apenas o preprocessamento de validação (normalização)
test_dataset = test_dataset.map(preprocess_val).prefetch(AUTOTUNE)

print("\n🧪 Avaliando no Dataset Externo...")
external_loss, external_acc = model.evaluate(test_dataset)
print(f"🏆 Acurácia no Teste Externo: {external_acc*100:.2f}%")


### 8.1 Visualização das Predições no Teste


In [ ]:
# Pega um batch de imagens
image_batch, label_batch = next(iter(test_dataset))
predictions = model.predict(image_batch)
predicted_classes = np.argmax(predictions, axis=1)

test_class_names = class_names # Classes salvas anteriormente

plt.figure(figsize=(14, 10))
for i in range(min(16, len(image_batch))):
    ax = plt.subplot(4, 4, i + 1)
    
    # Desnormaliza para mostrar
    img_show = (image_batch[i] + 1) / 2
    plt.imshow(img_show)
    
    true_label = test_class_names[label_batch[i]]
    pred_label = test_class_names[predicted_classes[i]]
    confidence = np.max(predictions[i]) * 100
    
    color = "green" if true_label == pred_label else "red"
    
    plt.title(f"Real: {true_label}\nPred: {pred_label}\n({confidence:.1f}%)", color=color, fontsize=10)
    plt.axis("off")
plt.suptitle("Predições no Dataset Externo", fontsize=16)
plt.show()


## 9. 💾 Exportação Otimizada


In [ ]:
model.save("mask_detector_light.keras")

# TFLite Conversion
print("\n🔄 Convertendo para TFLite INT8...")
def representative_data_gen():
    for input_value, _ in train_dataset.take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

tflite_path = "mask_detector_light_int8.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

size_kb = os.path.getsize(tflite_path) / 1024
print(f"✅ Modelo TFLite salvo: {tflite_path}")
print(f"📦 Tamanho Final: {size_kb:.2f} KB")

if size_kb < 400:
    print("🎉 SUCESSO! Meta de < 400KB atingida.")
else:
    print("⚠️ Ainda acima de 400KB. Tente reduzir input_shape ou alpha.")